# Обнаружение статистически значимых отличий в уровнях экспрессии генов больных раком
#### Это задание поможет вам лучше разобраться в методах множественной проверки гипотез и позволит применить ваши знания на данных из реального биологического исследования.

В этом задании вы:
вспомните, что такое t-критерий Стьюдента и для чего он применяется
сможете применить технику множественной проверки гипотез и увидеть собственными глазами, как она работает на реальных данных
почувствуете разницу в результатах применения различных методов поправки на множественную проверку
Основные библиотеки и используемые методы:
* Библиотека scipy и основные статистические функции:http://docs.scipy.org/doc/scipy/reference/stats.html#statistical-functions

* Библиотека statmodels для методов коррекции при множественном сравнении:

http://statsmodels.sourceforge.net/devel/stats.html

* Статья, в которой рассматриваются примеры использования statsmodels для множественной проверки гипотез:

http://jpktd.blogspot.ru/2013/04/multiple-testing-p-value-corrections-in.html

Описание используемых данных
Данные для этой задачи взяты из исследования, проведенного в Stanford School of Medicine. В исследовании была предпринята попытка выявить набор генов, которые позволили бы более точно диагностировать возникновение рака груди на самых ранних стадиях.

В эксперименте принимали участие 24 человек, у которых не было рака груди (normal), 25 человек, у которых это заболевание было диагностировано на ранней стадии (early neoplasia), и 23 человека с сильно выраженными симптомами (cancer).


Ученые провели секвенирование биологического материала испытуемых, чтобы понять, какие из этих генов наиболее активны в клетках больных людей.

Секвенирование — это определение степени активности генов в анализируемом образце с помощью подсчёта количества соответствующей каждому гену РНК.

В данных для этого задания вы найдете именно эту количественную меру активности каждого из 15748 генов у каждого из 72 человек, принимавших участие в эксперименте.

Вам нужно будет определить те гены, активность которых у людей в разных стадиях заболевания отличается статистически значимо.

Кроме того, вам нужно будет оценить не только статистическую, но и практическую значимость этих результатов, которая часто используется в подобных исследованиях.

Диагноз человека содержится в столбце под названием "Diagnosis".

Практическая значимость изменения

Цель исследований — найти гены, средняя экспрессия которых отличается не только статистически значимо, но и достаточно сильно. В экспрессионных исследованиях для этого часто используется метрика, которая называется fold change (кратность изменения). Определяется она следующим образом:

<img src=image.png>

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels

In [2]:
data = pd.read_csv('gene_high_throughput_sequencing.csv')
data.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


## Часть 1: применение t-критерия Стьюдента

В первой части вам нужно будет применить критерий Стьюдента для проверки гипотезы о равенстве средних в двух независимых выборках. Применить критерий для каждого гена нужно будет дважды:

* для групп normal (control) и early neoplasia (treatment)
* для групп early neoplasia (control) и cancer (treatment)

В качестве ответа в этой части задания необходимо указать количество статистически значимых отличий, которые вы нашли с помощью t-критерия Стьюдента, то есть число генов, у которых p-value этого теста оказался меньше, чем уровень значимости.

In [3]:
data_normal = data[data.Diagnosis == 'normal']
data_early_neoplasia = data[data.Diagnosis == 'early neoplasia']
data_cancer = data[data.Diagnosis == 'cancer']

In [4]:
data_normal.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


Примем, что гены по выборкам распределены нормально (критерий Шапиро-Уилка), для того чтобы использовать двухвыборочный критерий Стьюдента.
Пусть условие применимости (дисперсия выше у выборки с бОльшим числом элементов) также выполняется.

In [5]:
data_normal.drop(['Patient_id', 'Diagnosis'], axis = 1, inplace = True)
data_early_neoplasia.drop(['Patient_id', 'Diagnosis'], axis = 1, inplace = True)
data_cancer.drop(['Patient_id', 'Diagnosis'], axis = 1, inplace = True)
stat1, p1 = stats.ttest_ind(data_normal, data_early_neoplasia, equal_var = False)

/Users/andrewbelyaev/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [6]:
sum(p1<0.05)

1575

In [7]:
stat2, p2 = stats.ttest_ind(data_early_neoplasia, data_cancer, equal_var = False)

In [8]:
sum(p2<0.05)

3490

In [9]:
with open("ans1_1.txt", 'w') as outfile:
    outfile.write(str(sum(p1<0.05)))
with open("ans1_2.txt", 'w') as outfile:
    outfile.write(str(sum(p2<0.05)))

## Часть 2: поправка методом Холма

В этой части задания нужно будет применить поправку Холма для получившихся двух наборов достигаемых уровней значимости из предыдущей части. Обратите внимание, что поскольку вы будете делать поправку для каждого из двух наборов p-value отдельно, то проблема, связанная с множественной проверкой останется.

Для того, чтобы ее устранить, достаточно воспользоваться поправкой Бонферрони, то есть использовать уровень значимости 0.05 / 2 вместо 0.05 для дальнейшего уточнения значений p-value c помощью метода Холма.

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Холма-Бонферрони. Причем это число нужно ввести с учетом практической значимости: посчитайте для каждого значимого изменения fold change и выпишите в ответ число таких значимых изменений, абсолютное значение fold change которых больше, чем 1.5.

In [10]:
import statsmodels.stats.multitest as smm

/Users/andrewbelyaev/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [11]:
reject1, p_corrected1, a1, a2 = smm.multipletests(p1, alpha = 0.025, method = 'holm')

In [12]:
p_array1 = pd.DataFrame(data = {'p': p1, 'p_corrected': p_corrected1, 'reject': reject1})

In [13]:
def count_Fc(data1, data2, reject_true_indecies, k = 1.5):
    data1 = data1.mean()
    data2 = data2.mean()
    count = 0
    for index in reject_true_indecies:
        if data1[index] > data2[index]:
            if data1[index]/data2[index] > k:
                count += 1
        else:
            if data2[index]/data1[index] > k:
                count += 1
    return count

In [14]:
# for item in p_array1[p_array1.reject == True]:
#     if item.p > p_corrected:
ans2_1 = count_Fc(data_normal, data_early_neoplasia, p_array1[p_array1.reject == True].index)
p_array1[p_array1.reject == True]

,p,p_corrected,reject
7244,7.955435e-07,0.012527,True
9820,8.498742e-08,0.001338,True


In [15]:
reject2, p_corrected2, a1, a2 = smm.multipletests(p2, alpha = 0.025, method = 'holm')
p_array2 = pd.DataFrame(data = {'p': p2, 'p_corrected': p_corrected2, 'reject': reject2})
p_array2[p_array2.reject == True]

,p,p_corrected,reject
47,2.276043e-07,0.003575,True
283,1.087046e-07,0.001709,True
316,1.002085e-06,0.015708,True
1105,1.041679e-07,0.001637,True
1238,1.103311e-06,0.017292,True
...,...,...,...
15090,1.938708e-09,0.000031,True
15449,3.981097e-08,0.000626,True
15576,7.554205e-08,0.001188,True
15640,5.832350e-07,0.009152,True


In [16]:
p_p_corrected = np.array(p_array2[p_array2.reject == True][['p', 'p_corrected']]) 
ans2_2 = count_Fc(data_early_neoplasia, data_cancer, p_array2[p_array2.reject == True].index)
ans2_2

77

In [17]:
# Итого получаем 4 значимых гена при использовании поправка Холма-Бенферрони

In [18]:
with open("ans2_1.txt", 'w') as outfile:
    outfile.write(str(2))
with open("ans2_2.txt", 'w') as outfile:
    outfile.write(str(ans2_2))

## Часть 3: поправка методом Бенджамини-Хохберга


Данная часть задания аналогична второй части за исключением того, что нужно будет использовать метод Бенджамини-Хохберга.

Обратите внимание, что методы коррекции, которые контролируют FDR, допускает больше ошибок первого рода и имеют большую мощность, чем методы, контролирующие FWER. Большая мощность означает, что эти методы будут совершать меньше ошибок второго рода (то есть будут лучше улавливать отклонения от H_0
​	 , когда они есть, и будут чаще отклонять H_0
​	 , когда отличий нет).

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Бенджамини-Хохберга, причем так же, как и во второй части, считать только такие отличия, у которых abs(fold change) > 1.5.

In [19]:
reject1, p_corrected1, a1, a2 = smm.multipletests(p1, alpha = 0.025, method = 'fdr_bh')
p_array1 = pd.DataFrame(data = {'p': p1, 'p_corrected': p_corrected1, 'reject': reject1})
p_array1[p_array1.reject == True]

,p,p_corrected,reject
4627,2.222228e-06,0.008749,True
7244,7.955435e-07,0.006264,True
8250,2.015472e-06,0.008749,True
9820,8.498742e-08,0.001338,True


In [20]:
ans3_1 = count_Fc(data_normal, data_early_neoplasia, p_array1[p_array1.reject == True].index)
ans3_1

4

In [21]:
reject2, p_corrected2, a1, a2 = smm.multipletests(p2, alpha = 0.025, method = 'fdr_bh')
p_array2 = pd.DataFrame(data = {'p': p2, 'p_corrected': p_corrected2, 'reject': reject2})
p_array2[p_array2.reject == True]

,p,p_corrected,reject
8,7.402400e-04,0.017556,True
17,2.945987e-04,0.009487,True
44,1.154858e-03,0.022992,True
47,2.276043e-07,0.000086,True
54,5.973956e-04,0.015198,True
...,...,...,...
15654,4.875591e-04,0.013193,True
15692,1.779494e-04,0.007186,True
15694,2.069987e-04,0.007799,True
15703,6.795085e-08,0.000045,True


In [22]:
p_p_corrected = np.array(p_array2[p_array2.reject == True][['p', 'p_corrected']]) 
ans3_2 = count_Fc(data_early_neoplasia, data_cancer, p_array2[p_array2.reject == True].index)
ans3_2

524

In [23]:
with open("ans3_1.txt", 'w') as outfile:
    outfile.write(str(4))
with open("ans3_2.txt", 'w') as outfile:
    outfile.write(str(ans3_2))